# Day 11

# 문서 전처리

모든 데이터 분석 모형은 숫자로 구성된 고정 차원 벡터를 독립 변수로 하고 있으므로 문서(document)를 분석을 하는 경우에도 숫자로 구성된 특징 벡터(feature vector)를 문서로부터 추출하는 과정이 필요하다. 이러한 과정을 문서 전처리(document preprocessing)라고 한다.

## BOW (Bag of Words)

문서를 숫자 벡터로 변환하는 가장 기본적인 방법은 BOW (Bag of Words) 이다. BOW 방법에서는 전체 문서 $\{D_1, D_2, \ldots, D_n\}$ 를 구성하는 고정된 단어장(vocabulary) $\{W_1, W_2, \ldots, W_m\}$ 를  만들고 $D_i$라는 개별 문서에 단어장에 해당하는 단어들이 포함되어 있는지를 표시하는 방법이다.

$$ \text{ 만약 단어 } W_j \text{가 문서} D_i \text{ 안에 있으면 }, \;\; \rightarrow x_{ij} = 1 $$ 

## Scikit-Learn 의 문서 전처리 기능

Scikit-Learn 의 feature_extraction.text 서브 패키지는 다음과 같은 문서 전처리용 클래스를 제공한다.

* [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html): 
 * 문서 집합으로부터 단어의 수를 세어 카운트 행렬을 만든다.
* [`TfidfVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html): 
 * 문서 집합으로부터 단어의 수를 세고 TF-IDF 방식으로 단어의 가중치를 조정한 카운트 행렬을 만든다.
* [`HashingVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html): 
 * hashing trick 을 사용하여 빠르게 카운트 행렬을 만든다.
 

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',    
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'and': 0,
 'document': 1,
 'first': 2,
 'is': 3,
 'last': 4,
 'one': 5,
 'second': 6,
 'the': 7,
 'third': 8,
 'this': 9}

In [32]:
pd.Series(vect.vocabulary_)

and         0
document    1
first       2
is          3
last        4
one         5
second      6
the         7
third       8
this        9
dtype: int64

In [33]:
vect.transform(['This is the second document.']).toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [34]:
vect.transform(['Something completely new.']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [35]:
vect.transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

## 문서 처리 옵션

`CountVectorizer`는 다양한 인수를 가진다. 그 중 중요한 것들은 다음과 같다.

* `stop_words` : 문자열 {‘english’}, 리스트 또는 None (디폴트)
 * stop words 목록.‘english’이면 영어용 스탑 워드 사용.
* `analyzer` : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수
 * 단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램 
* `tokenizer` : 함수 또는 None (디폴트)
 * 토큰 생성 함수 .
* `token_pattern` : string
 * 토큰 정의용 정규 표현식 
* `ngram_range` : (min_n, max_n) 튜플
 * n-그램 범위 
* `max_df` : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
 * 단어장에 포함되기 위한 최대 빈도
* `min_df` : 정수 또는 [0.0, 1.0] 사이의 실수.  디폴트 1
 * 단어장에 포함되기 위한 최소 빈도 
* `vocabulary` : 사전이나 리스트
 * 단어장

## Stop Words

Stop Words 는 문서에서 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 보통 영어의 관사나 접속사, 한국어의 조사 등이 여기에 해당한다. `stop_words` 인수로 조절할 수 있다.

In [36]:
vect = CountVectorizer(stop_words=["and", "is", "the", "this"]).fit(corpus)
vect.vocabulary_

{'document': 0, 'first': 1, 'last': 2, 'one': 3, 'second': 4, 'third': 5}

In [37]:
vect = CountVectorizer(stop_words="english").fit(corpus)
vect.vocabulary_

{'document': 0, 'second': 1}

## 토큰(token)

토큰은 문서에서 단어장을 생성할 때 하나의 단어가 되는 단위를 말한다. `analyzer`, `tokenizer`, `token_pattern` 등의 인수로 조절할 수 있다.

In [38]:
vect = CountVectorizer(analyzer="char").fit(corpus)
vect.vocabulary_

{' ': 0,
 '.': 1,
 '?': 2,
 'a': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'h': 8,
 'i': 9,
 'l': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'r': 14,
 's': 15,
 't': 16,
 'u': 17}

In [39]:
vect = CountVectorizer(token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'the': 0, 'third': 1, 'this': 2}

In [40]:
import nltk
nltk.download("punkt")
vect = CountVectorizer(tokenizer=nltk.word_tokenize).fit(corpus)
vect.vocabulary_

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jihoon_Kim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


{'.': 0,
 '?': 1,
 'and': 2,
 'document': 3,
 'first': 4,
 'is': 5,
 'last': 6,
 'one': 7,
 'second': 8,
 'the': 9,
 'third': 10,
 'this': 11}

## n-그램

n-그램은 단어장 생성에 사용할 토큰의 크기를 결정한다. 1-그램은 토큰 하나만 단어로 사용하며 2-그램은 두 개의 연결된 토큰을 하나의 단어로 사용한다.

In [41]:
vect = CountVectorizer(ngram_range=(2,2)).fit(corpus)
vect.vocabulary_

{'and the': 0,
 'first document': 1,
 'is the': 2,
 'is this': 3,
 'last document': 4,
 'second document': 5,
 'second second': 6,
 'the first': 7,
 'the last': 8,
 'the second': 9,
 'the third': 10,
 'third one': 11,
 'this is': 12,
 'this the': 13}

In [42]:
vect = CountVectorizer(ngram_range=(1,2), token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'the': 0, 'the third': 1, 'third': 2, 'this': 3, 'this the': 4}

## 빈도수

`max_df`, `min_df` 인수를 사용하여 문서에서 토큰이 나타난 횟수를 기준으로 단어장을 구성할 수도 있다. 토큰의 빈도가 `max_df`로 지정한 값을 초과 하거나 `min_df`로 지정한 값보다 작은 경우에는 무시한다. 인수 값은 정수인 경우 횟수, 부동소수점인 경우 비중을 뜻한다. 

In [43]:
vect = CountVectorizer(max_df=4, min_df=2).fit(corpus)
vect.vocabulary_, vect.stop_words_

({'document': 0, 'first': 1, 'is': 2, 'this': 3},
 {'and', 'last', 'one', 'second', 'the', 'third'})

In [44]:
vect.transform(corpus).toarray().sum(axis=0)

array([4, 2, 3, 3], dtype=int64)

## TF-IDF

TF-IDF(Term Frequency – Inverse Document Frequency) 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법이다. 


구제적으로는 문서 $d$(document)와 단어 $t$ 에 대해 다음과 같이 계산한다.

$$ \text{tf-idf}(d, t) = \text{tf}(d, t) \cdot \text{idf}(t) $$


여기에서

* $\text{tf}(d, t)$: 단어의 빈도수
* $\text{idf}(t)$ : inverse document frequency 
 
 $$ \text{idf}(t) = \log \dfrac{n_d}{1 + \text{df}(t)} $$
 
* $n_d$ : 전체 문서의 수
* $\text{df}(t)$:  단어 $t$를 가진 문서의 수

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
tfidv = TfidfVectorizer().fit(corpus)
tfidv.transform(corpus).toarray()

array([[ 0.        ,  0.38947624,  0.55775063,  0.4629834 ,  0.        ,
         0.        ,  0.        ,  0.32941651,  0.        ,  0.4629834 ],
       [ 0.        ,  0.24151532,  0.        ,  0.28709733,  0.        ,
         0.        ,  0.85737594,  0.20427211,  0.        ,  0.28709733],
       [ 0.55666851,  0.        ,  0.        ,  0.        ,  0.        ,
         0.55666851,  0.        ,  0.26525553,  0.55666851,  0.        ],
       [ 0.        ,  0.38947624,  0.55775063,  0.4629834 ,  0.        ,
         0.        ,  0.        ,  0.32941651,  0.        ,  0.4629834 ],
       [ 0.        ,  0.45333103,  0.        ,  0.        ,  0.80465933,
         0.        ,  0.        ,  0.38342448,  0.        ,  0.        ]])

## Hashing Trick

`CountVectorizer`는 모든 작업을 메모리 상에서 수행하므로 처리할 문서의 크기가 커지면 속도가 느려지거나 실행이 불가능해진다. 이 때  `HashingVectorizer`를 사용하면 해시 함수를 사용하여 단어에 대한 인덱스 번호를 생성하기 때문에 메모리 및 실행 시간을 줄일 수 있다.

In [47]:
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups()
len(twenty.data)

11314

In [48]:
%time CountVectorizer().fit(twenty.data).transform(twenty.data)

Wall time: 5.92 s


<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [49]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=10)

In [50]:
%time hv.transform(twenty.data)

Wall time: 3.08 s


<11314x10 sparse matrix of type '<class 'numpy.float64'>'
	with 112863 stored elements in Compressed Sparse Row format>

## 형태소 분석기 이용

In [51]:
corpus = ["imaging", "image", "imagination", "imagine", "buys", "buying", "bought"]
vect = CountVectorizer().fit(corpus)
vect.vocabulary_

{'bought': 0,
 'buying': 1,
 'buys': 2,
 'image': 3,
 'imagination': 4,
 'imagine': 5,
 'imaging': 6}

In [52]:
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups()
docs = twenty.data[:100]

In [53]:
vect = CountVectorizer(stop_words="english", token_pattern="wri\w+").fit(docs)
vect.vocabulary_

{'write': 0,
 'writer': 1,
 'writers': 2,
 'writes': 3,
 'writing': 4,
 'writing_': 5,
 'written': 6}

In [54]:
from nltk.stem import SnowballStemmer

class StemTokenizer(object):
    def __init__(self):
        self.s = SnowballStemmer('english')
        self.t = CountVectorizer(stop_words="english", token_pattern="wri\w+").build_tokenizer()
    def __call__(self, doc):
        return [self.s.stem(t) for t in self.t(doc)]

vect = CountVectorizer(tokenizer=StemTokenizer()).fit(docs)
vect.vocabulary_

{'write': 0, 'writer': 1, 'writing_': 2, 'written': 3}

# Python을 사용한 웹 서비스 개요

Python은 현재 웹 서비스 부분에서 활발히 사용되고 있는 언어의 하나이다. 파이썬을 사용하여 웹서비스를 구축하는 경우, 보통 다음과 같은 구조를 가지게 된다.

<img src="https://datascienceschool.net/upfiles/015a1deab591474a80637365668a8bd6.png" style="width:100%;">


## 리버스 프락시 서버

일반적으로 보안 문제나 정적 파일(static file)의 빠른 서비스, 로드 밸런싱 등을 위해 리버스 프락시(reverse proxy) 서버를 실제 웹 서비스 서버 앞단에 두는 경우가 많다. 리버스 프락시 서버는 클라이언트로부터의 요청을 실제 웹서버나 파일서버로 전달하거나 대리하는 역할을 한다.

많이 쓰이는 리버스 프락시 서버로는 nginx 등이 있다.

## WSGI 웹 어플리케이션 서버

웹 어플리케이션 서버(Web Application Server)는 실제로 클라이언트 호출에 대응하여 HTML 파일 등의 결과물을 출력하는 역할을 하는 서버 프로세스이다. 

클라이언트 호출은 원격에서 이루어지는 함수 호출에 비유할 수 있다. 실제로 함수 호출을 할 수 있는 프로세스는 웹 어플리케이션 서버이지만 함수 자체는 웹 어플리케이션(Web Application)이라고 부르는 일종의 라이브러리와 같은 형태로 구현되어 웹 어플리케이션 서버가 해당 어플리케이션(라이브러리)를 임포트하여 사용한다.

Python의 경우에는 WSGI(Web Server Gateway Interface)이라고 부르는 웹 어플리케이션 규약이 존재하여 모든 웹 어플리케이션은 WSGI 규약에 맞게 구현되어야 한다.

웹 어플리케이션 서버로는 Apache에 modwsgi 모듈을 추가하여 사용할 수 있지만 Python으로 만들어진 웹 어플리케이션을 임포트하기 때문에 웹 어플리케이션 서버도 Python인 경우가 많다. tornado, uwsgi, gnuicorn, werkzeug, twisted 이 많이 사용되고 있다. 

* 웹 어플리케이션 서버의 성능 비교: http://nichol.as/benchmark-of-python-web-servers

## WSGI 웹 어플리케이션
WSGI: 파이썬에서 사용하는 Web application server와 web application이 통신하는 규약임.

WSGI 웹 어플리케이션은 실제로 다음과 같이 environment(또는 context)와 request 인수를 받아서 response 를 출력하는 함수의 집합이다.

웹 서비스의 각 URL은 이 함수들에 매핑(mapping)되어 웹 어플리케이션 서버가 웹 브라우저의 요청을 받을 때마다 해당 함수를 찾아서 호출하게 된다. 실제 함수 매핑은 웹 어플리케이션 서버보다는 웹 어플리케이션 자체(혹은 웹 어플리케이션 프레임워크)에서 발생하는 경우가 많다.

웹 어플리케이션은 독자적으로 함수 패키지를 만들기 보다는 웹 어플리케이션 프레임워크라고 불리는 구조 및 기반 클래스를 사용하여 만드는 경우가 대부분이다.

## 웹 어플리케이션 프레임워크

웹 어플리케이션 프레임워크는 WSGI 웹 어플리케이션을 빠르고 쉽게 개발하기 위한 기반 구조(archtecture) 및 클래스 라이브러리를 말한다.

Python으로 구현된 다양한 웹 어플리케이션 프레임워크들이 존재한다. 일부 예를 들면 다음과 같다.

* django
* flask
* pyramids
* turbogears
* weppy
* web2py
* bottle
* falcon
* muffin
* pylon
* grokk
* zope



* 웹 어플리케이션 프레임워크 성능 비교: http://klen.github.io/py-frameworks-bench/

웹 어플리케이션의 주 기능 중의 하나는 URL과 함수간의 매핑이다. 예를 들어 django의 경우 다음과 같은 URL 설정을 사용하는데

```python
urlpatterns = [
    url(r'^$', views.index, name='index'),
    url(r'^(?P<question_id>[0-9]+)/$', views.detail, name='detail'),
]
```

이 설정에 따르면 웹 브라우저에서
* http://server/polls/ 를 호출하면 `views` 패키지의 `index` 함수가 호출되고 
* http://server/polls/5/ 를 호출하면 `views` 패키지의 `detail` 함수가 `question_id` 인수 5와 함께 호출된다.

## ORM (Object Relation Mapper)

웹 서비스가 하는 대부분의 일은 데이터 베이스로부터 특정한 데이터를 찾아 보여주는 작업이다. 따라서 데이터 베이스를 쉽게 다울 수 있는 도구가 필수적이다. 

ORM은 데이터 베이스의 구조 즉, 스키마를 Python 클래스 정의로 구현할 수 있도록 한다. 예를 들어 django의 경우 자체적인 ORM을 사용하는데 다음과 같이 클래스를 정의하게 되면

```python
class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')
```

데이터 베이스에서 스키마를 지정하여 테이블을 생성하는 SQL을 자동 생성하여 실행시킨다.

```
CREATE TABLE "question" (
    "id" serial NOT NULL PRIMARY KEY,
    "question_text" varchar(200) NOT NULL,
    "pub_date" timestamp with time zone NOT NULL
);
```





## Template Engine

데이터 베이스로부터 얻어진 데이터는 json이나 txt 같은 단순한 형태로 서비스되기도 하지만 대부분 html 파일을 형태로 서비스된다. 따라서 데이터 요소를 포함하는 html 파일을 동적으로 생성하는 기능을 필요하다. Template Engine은 미리 만들어진 html 파일 template의 일부 문자열을 실제 데이터로 치환(replace)하는 역할을 한다.

django의 경우에는 자체 template engine을 가지고 있지만 flask는 jinja2와 같은 외부 파이썬 패키지를 사용한다. 

예를 들어 jinja2 패키지는 다음과 같은 context와 

```
contries = [
{'Name': 'Afghanistan', 'Population': 22720000},
{'Name': 'Albania', 'Population': 3401200},
{'Name': 'Algeria', 'Population': 31471000},
]
```

다음 template을 합성하여 

```
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<title>{{ title }}</title>
</head>
<body>
<table>
{% for country in countries %}
<tr><td>{{ country['Name'] }}</td><td>{{ country['Population'] }}</td></tr>
{% endfor %}
</table>
</body>
</html>
```

다음과 같은 html 을 생성할 수 있다.

```
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<title>Country list</title>
</head>
<body>
<table>
<tr><td>Afghanistan</td><td>22720000</td></tr>
<tr><td>Albania</td><td>3401200</td></tr>
<tr><td>Algeria</td><td>31471000</td></tr>
</table>
</body>
</html>
```

# Python을 활용한 웹 정보 수집

## urllib 패키지

* `urlencode` : URL 인수 문자열 생성
* `urlopen` : 웹서버 연결
* `urlretrieve` : 웹서버 연결 및 HTML 문서 저장

In [63]:
import urllib

In [64]:
urlstr = "http://www.google.com/finance/historical?q=NASDAQ%3AAAPL&output=csv"

In [65]:
urlobj = urllib.parse.urlparse(urlstr)
urlobj

ParseResult(scheme='http', netloc='www.google.com', path='/finance/historical', params='', query='q=NASDAQ%3AAAPL&output=csv', fragment='')

In [68]:
urllib.parse.parse_qs(urlobj.query)

{'output': ['csv'], 'q': ['NASDAQ:AAPL']}

In [78]:
symbol = "NASDAQ:NVDA"

In [79]:
urllib.parse.quote(symbol)

'NASDAQ%3ANVDA'

In [80]:
url = "http://www.google.com/finance/historical?q={}&output=csv".format(symbol)
url

'http://www.google.com/finance/historical?q=NASDAQ:NVDA&output=csv'

In [87]:
data = urllib.request.urlopen(url).read()
print(data[:1000])

b'\xef\xbb\xbfDate,Open,High,Low,Close,Volume\n31-May-17,146.69,147.00,142.05,144.35,22182894\n30-May-17,143.70,146.29,143.05,144.87,24741298\n26-May-17,137.93,145.28,137.11,141.84,19478497\n25-May-17,140.00,140.03,136.44,138.26,15205701\n24-May-17,140.96,141.07,138.08,138.57,20434495\n23-May-17,139.70,139.79,135.71,137.03,17031247\n22-May-17,137.77,139.48,137.33,138.90,20915225\n19-May-17,137.02,138.22,135.22,136.00,25459271\n18-May-17,129.50,133.43,127.05,133.07,28900757\n17-May-17,134.10,134.86,127.55,127.72,31784158\n16-May-17,136.38,137.44,133.36,136.81,28017471\n15-May-17,129.56,134.41,129.38,134.31,27188548\n12-May-17,126.63,129.60,125.78,127.89,24065459\n11-May-17,120.05,130.43,119.91,126.50,48494808\n10-May-17,114.29,121.82,114.02,121.29,53227434\n9-May-17,103.00,104.93,102.66,102.94,21191792\n8-May-17,104.34,104.40,102.31,102.77,9130990\n5-May-17,103.38,104.15,102.75,103.86,5710736\n4-May-17,104.50,104.95,103.53,103.85,5244692\n3-May-17,103.20,104.64,102.60,104.25,8422633\n2-

In [89]:
urllib.request.urlretrieve(url)

('C:\\Users\\JIHOON~1\\AppData\\Local\\Temp\\tmp4kyokvxd',
 <http.client.HTTPMessage at 0x151cce035c0>)

## requests 패키지

* http://docs.python-requests.org/en/master/
* HTTP protocols (get, post, put, delete, head, options)


https://www.google.com/finance/historical?q=KRX%3AKOSPI200

In [90]:
import requests
url = "https://www.google.com/finance/historical?q=KRX%3AKOSPI200"
req = requests.get(url)
print(req.text[:1000])

<!DOCTYPE html><html><head><script>(function(){(function(){function e(a){this.t={};this.tick=function(a,c,b){var d=void 0!=b?b:(new Date).getTime();this.t[a]=[d,c];if(void 0==b)try{window.console.timeStamp("CSI/"+a)}catch(h){}};this.tick("start",null,a)}var a;if(window.performance)var d=(a=window.performance.timing)&&a.responseStart;var f=0<d?new e(d):new e;window.jstiming={Timer:e,load:f};if(a){var c=a.navigationStart;0<c&&d>=c&&(window.jstiming.srt=d-c)}if(a){var b=window.jstiming.load;0<c&&d>=c&&(b.tick("_wtsrt",void 0,c),b.tick("wtsrt_","_wtsrt",
d),b.tick("tbsd_","wtsrt_"))}try{a=null,window.chrome&&window.chrome.csi&&(a=Math.floor(window.chrome.csi().pageT),b&&0<c&&(b.tick("_tbnd",void 0,window.chrome.csi().startE),b.tick("tbnd_","_tbnd",c))),null==a&&window.gtbExternal&&(a=window.gtbExternal.pageT()),null==a&&window.external&&(a=window.external.pageT,b&&0<c&&(b.tick("_tbnd",void 0,window.external.startE),b.tick("tbnd_","_tbnd",c))),a&&(window.jstiming.pt=a)}catch(g){}})();}).cal

## beautifulsoup 패키지

* https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* HTML 문서 파싱 및 태그 검색

In [91]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(req.text, 'lxml')

In [92]:
table = soup.find("table", class_="gf-table historical_price")
print(table.prettify()[:600])

<table class="gf-table historical_price">
 <tr class="bb">
  <th class="bb lm lft">
   Date
  </th>
  <th class="rgt bb">
   Open
  </th>
  <th class="rgt bb">
   High
  </th>
  <th class="rgt bb">
   Low
  </th>
  <th class="rgt bb">
   Close
  </th>
  <th class="rgt bb rm">
   Volume
  </th>
 </tr>
 <tr>
  <td class="lm">
   May 31, 2017
  </td>
  <td class="rgt">
   303.79
  </td>
  <td class="rgt">
   306.18
  </td>
  <td class="rgt">
   303.64
  </td>
  <td class="rgt">
   304.67
  </td>
  <td class="rgt rm">
   114,684,000
  </td>
 </tr>
 <tr>
  <td class="lm">
   May 30, 2017
  </td>
  


In [93]:
table.find_all('tr')[2].find_all('td')

[<td class="lm">May 30, 2017
 </td>, <td class="rgt">306.89
 </td>, <td class="rgt">306.99
 </td>, <td class="rgt">303.11
 </td>, <td class="rgt">304.59
 </td>, <td class="rgt rm">94,122,000
 </td>]

In [94]:
import dateutil

list_records = []

for i, r in enumerate(table.find_all('tr')):
    record = None
    for j, c in enumerate(r.find_all('td')):
        if j == 0:
            record = {"date": dateutil.parser.parse(c.text.strip())}
        elif j == 1:
            record.update({"open": float(c.text.strip())})
        elif j == 2:
            record.update({"high": float(c.text.strip())})
        elif j == 3:
            record.update({"low": float(c.text.strip())})
        elif j == 4:
            record.update({"close": float(c.text.strip())})
        elif j == 5:
            record.update({"volume": int(c.text.strip().replace(',',''))})
    if record is not None:
        list_records.append(record)

In [95]:
list_records[0]

{'close': 304.67,
 'date': datetime.datetime(2017, 5, 31, 0, 0),
 'high': 306.18,
 'low': 303.64,
 'open': 303.79,
 'volume': 114684000}

In [96]:
df = pd.DataFrame(list_records, columns=["date", "open", "high", "low", "close", "volume"])
df.tail()

,date,open,high,low,close,volume
25,2017-04-20,276.32,278.04,275.75,277.76,69505000
26,2017-04-19,277.66,278.07,275.97,276.49,65259000
27,2017-04-18,279.36,279.42,277.05,278.23,54714000
28,2017-04-17,278.06,279.06,277.49,278.10,47713000
29,2017-04-14,277.59,278.14,276.12,277.31,44577000


## lxml 패키지

* http://lxml.de/index.html
* xpath 사용 가능
 * https://en.wikipedia.org/wiki/XPath
 * https://www.w3schools.com/xml/xpath_intro.asp

In [99]:
import lxml.html
import numpy as np
tree = lxml.html.fromstring(req.text)

In [100]:
import dateutil
dates = [dateutil.parser.parse(x.text.strip()) for x in tree.xpath('//td[@class="lm"]')]
prices = np.reshape([float(x.text.strip()) for x in tree.xpath('//td[@class="rgt"]')], (-1, 4))
volumes = np.array([int(x.text.strip().replace(',','')) for x in tree.xpath('//td[@class="rgt rm"]')])

In [101]:
dates[:5]

[datetime.datetime(2017, 5, 31, 0, 0),
 datetime.datetime(2017, 5, 30, 0, 0),
 datetime.datetime(2017, 5, 29, 0, 0),
 datetime.datetime(2017, 5, 26, 0, 0),
 datetime.datetime(2017, 5, 25, 0, 0)]

In [102]:
prices[:5]

array([[ 303.79,  306.18,  303.64,  304.67],
       [ 306.89,  306.99,  303.11,  304.59],
       [ 307.98,  309.32,  305.13,  306.52],
       [ 305.41,  308.51,  305.07,  306.96],
       [ 302.83,  305.34,  302.18,  305.22]])

In [103]:
price_o = prices[:,0]
price_h = prices[:,1]
price_l = prices[:,2]
price_c = prices[:,3]

df = pd.DataFrame({"date": dates, "open": price_o, "high": price_h, "low": price_l, "close": price_c, "volume": volumes},
                  columns=["date", "open", "high", "low", "close", "volume"])
df.tail()

,date,open,high,low,close,volume
25,2017-04-20,276.32,278.04,275.75,277.76,69505000
26,2017-04-19,277.66,278.07,275.97,276.49,65259000
27,2017-04-18,279.36,279.42,277.05,278.23,54714000
28,2017-04-17,278.06,279.06,277.49,278.10,47713000
29,2017-04-14,277.59,278.14,276.12,277.31,44577000


# Scrapy를 사용한 웹 크롤링

Scrapy는 다수의 프로세스를 동시에 가동하여 크롤링 효율을 높이고 데이터 베이스에 기록까지 할 수 있는 웹 크롤링용 파이썬 패키지이다.

* http://doc.scrapy.org/en/latest/index.html


Scrapy를 이용한 웹 크롤링 어플리케이션은 다음과 같은 순서로 개발한다.

* Scrapy shell을 이용한 문서 구조 파악
* Scrapy 프로젝트 생성
* Spider 클래스 구현
* Item 클래스 구현
* Pipeline 구현
* Setting 설정


## Scrapy shell

scrapy shell은 콘솔에서 실행가능한 shell 도구이다. 크롤링하고자 하는 웹사이트 url을 인수로 가진다. 예를 들어 https://www.google.com/finance/historical?q=KRX%3AKOSPI200 페이지를 접근하려면 다음과 같이 실행한다.

```python
$ scrapy shell https://www.google.com/finance/historical?q=KRX%3AKOSPI200
```

실행하면 다음과 같은 페이지가 나타나며 ipython 콘솔이 실행된다.


```
2016-07-07 08:28:13 [scrapy] INFO: Scrapy 1.0.3 started (bot: scrapybot)
2016-07-07 08:28:13 [scrapy] INFO: Optional features available: ssl, http11, boto
2016-07-07 08:28:13 [scrapy] INFO: Overridden settings: {'LOGSTATS_INTERVAL': 0}
2016-07-07 08:28:13 [scrapy] INFO: Enabled extensions: CloseSpider, TelnetConsole, CoreStats, SpiderState
2016-07-07 08:28:13 [boto] DEBUG: Retrieving credentials from metadata server.
2016-07-07 08:28:13 [scrapy] INFO: Enabled downloader middlewares: HttpAuthMiddleware, DownloadTimeoutMiddleware, UserAgentMiddleware, RetryMiddleware, DefaultHeadersMiddleware, MetaRefreshMiddleware, HttpCompressionMiddleware, RedirectMiddleware, CookiesMiddleware, ChunkedTransferMiddleware, DownloaderStats
2016-07-07 08:28:13 [scrapy] INFO: Enabled spider middlewares: HttpErrorMiddleware, OffsiteMiddleware, RefererMiddleware, UrlLengthMiddleware, DepthMiddleware
2016-07-07 08:28:13 [scrapy] INFO: Enabled item pipelines:
2016-07-07 08:28:13 [scrapy] DEBUG: Telnet console listening on 127.0.0.1:6023
2016-07-07 08:28:13 [scrapy] INFO: Spider opened
2016-07-07 08:28:14 [scrapy] DEBUG: Crawled (200) <GET https://www.google.com/finance/historical?q=KRX%3AKOSPI200> (referer: None)
[s] Available Scrapy objects:
[s]   crawler    <scrapy.crawler.Crawler object at 0x7fcd973f0d90>
[s]   item       {}
[s]   request    <GET https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
[s]   response   <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
[s]   settings   <scrapy.settings.Settings object at 0x7fcd94f80b90>
[s]   spider     <DefaultSpider 'default' at 0x7fcd8d9b2f50>
[s] Useful shortcuts:
[s]   shelp()           Shell help (print this help)
[s]   fetch(req_or_url) Fetch request (or URL) and update local objects
[s]   view(response)    View response in a browser
2016-07-07 08:28:14 [root] DEBUG: Using default logger
2016-07-07 08:28:14 [root] DEBUG: Using default logger

In [1]:

```

이 ipython 콘솔은 다음과 같은 객체들을 이미 생성해 놓은 상태이다. 웹서버의 응답 즉, 웹페이지 내용은 `response` 객체에 저장되어 있다.

* `crawler`
* `request`
* `response`



```
In [1]: type(response)
Out[1]: scrapy.http.response.html.HtmlResponse

In [2]: response.url
Out[2]: 'https://www.google.com/finance/historical?q=KRX%3AKOSPI200'

In [3]: response.body[:100]
Out[3]: '<!DOCTYPE html><html><head><script>(function(){(function(){function e(a){this.t={};this.tick=functio'

```

 `response` 객체 즉, `scrapy.http.response.html.HtmlResponse` 클래스는 HTML 파싱을 위한 `xpath` 등의 메서드를 제공한다. 이를 이용하면 원하는 html 요소를 선택할 수 있다.
 
 * http://doc.scrapy.org/en/latest/topics/request-response.html?#response-objects

```
In [4]: response.xpath('//td[@class="lm"]')
Out[4]:
[<Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jul 6, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jul 5, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jul 4, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jul 1, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 30, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 29, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 28, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 27, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 24, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 23, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 22, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 21, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 20, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 17, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 16, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 15, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 14, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 13, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 10, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 9, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 8, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 7, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 3, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 2, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">Jun 1, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">May 31, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">May 30, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">May 27, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">May 26, 2016\n</td>'>,
 <Selector xpath='//td[@class="lm"]' data=u'<td class="lm">May 25, 2016\n</td>'>]

```

## Scrapy 프로젝트 생성

scrapy shell 을 사용하여 원하는 요소에 대한 조사가 끝나면 실제로 scrapy 를 구현해야 한다. 첫번재 단계로 프로젝트를 생성한다.

```
scrapy startproject tutorial
```

이 `tutorial` 프로젝트를 담고 있는 다음과 같이 디렉토리가 생성된다

```
tutorial/
    scrapy.cfg            # deploy configuration file
    tutorial/             # project's Python module, you'll import your code from here
        __init__.py
        items.py          # project items file
        pipelines.py      # project pipelines file
        settings.py       # project settings file
        spiders/          # a directory where you'll later put your spiders
            __init__.py
```


## Spider 클래스 구현

`spiders` 디렉토리 아래에는 실제로 웹 페이지를 읽고 데이터를 반환하는 클래스를 구현한다.


```python

# __init__.py

from dailystock import *

```

```python
# dailystock.py

import scrapy
import numpy as np
from dateutil.parser import parse

class DailyStockSpider(scrapy.Spider):
    name = "dailystock"
    start_urls = ["https://www.google.com/finance/historical?q=KRX%3AKOSPI200"]

    def parse(self, response):
        dates = [parse(x.extract().strip()) for x in response.xpath('//td[@class="lm"]/text()')]
        volumes = np.array([int(x.extract().strip().replace(',','')) for x in response.xpath('//td[@class="rgt rm"]/text()')])
        prices = np.reshape([float(x.extract().strip()) for x in response.xpath('//td[@class="rgt"]/text()')], (-1, 4))
        for d, v, p in zip(dates, volumes, prices):
          symbol = "KOSPI"
          date = d
          price_open = p[0]
          price_high = p[1]
          price_low = p[2]
          price_close = p[3]
          volume = v
          yield {"symbol": symbol, "date": date, 
                 "price_open": price_open, "price_high": price_high, 
                 "price_low": price_low, "price_close": price_close, 
                 "volume": volume}
```

일단 spider가 구현되면 다음과 같이 크롤링을 할 수 있다. 이 명령은 프로젝트 디렉토리 아래에서 실행해야 한다.

```
scrapy crawl dailystock -o data.json
```

## Item 클래스 구현

`items.py` 파일내에는 데이터베이스 레코드를 구현한다.

```
import scrapy

class DailyStockItem(scrapy.Item):
    symbol = scrapy.Field()
    date = scrapy.Field()
    price_open = scrapy.Field()
    price_high = scrapy.Field()
    price_low = scrapy.Field()
    price_close = scrapy.Field()
    volume = scrapy.Field()
```

## Pipeline 구현

pipeline은 수집한 데이터를 파일이 아닌 데이터베이스에 직접 넣기 위한 것이다. `pipelines.py` 파일에 구현한다. 보통 생성자에서 데이터베이스 연결을 만들고 `process_item` 메서드에서 레코드 입력 및 커밋(commit)을 한다. 

여기에서는 sqlite 데이터베이스를 사용하였다.


```python
import sqlite3
import os


class DailyStockPipeline(object):
    filename = 'dailystock.sqlite'

    def __init__(self):
        self.conn = None
        if os.path.exists(self.filename):
            self.conn = sqlite3.connect(self.filename)
        else:
            self.conn = sqlite3.connect(self.filename)
            self.conn.execute("""create table dailystock
                (symbol TEXT NOT NULL,
                 date TIMESTAMP NOT NULL,
                 price_open REAL,
                 price_high REAL,
                 price_low REAL,
                 price_close REAL,
                 volume INTEGER,
                 PRIMARY KEY (symbol, date))""")
            self.conn.commit()

    def process_item(self, item, domain):
        try:
            self.conn.execute('insert into dailystock values(?,?,?,?,?,?,?)',
                (item['symbol'], item['date'],
                 item['price_open'], item['price_high'],
                 item['price_low'], item['price_close'],
                 item['volume']))
            self.conn.commit()
        except Exception, e:
            print str(e)
        return item
```

## Setting 설정

이 pipeline을 사용하기 위해서는 settings.py 파일에 다음과 같이 설정을 추가해야 한다.

```python
BOT_NAME = 'tutorial'
SPIDER_MODULES = ['tutorial.spiders']
NEWSPIDER_MODULE = 'tutorial.spiders'
ITEM_PIPELINES = {
    'tutorial.pipelines.DailyStockPipeline': 300,
}
DOWNLOAD_HANDLERS = {
    's3': None,
}
```









## 크롤링 

실제로 크롤링을 하려면 tutorial 프로젝트 디렉토리에서 다음과 같이 명령한다.

```
$ scrapy crawl dailystock
2016-07-08 03:40:40 [scrapy] INFO: Scrapy 1.0.3 started (bot: tutorial)
2016-07-08 03:40:40 [scrapy] INFO: Optional features available: ssl, http11, boto
2016-07-08 03:40:40 [scrapy] INFO: Overridden settings: {'NEWSPIDER_MODULE': 'tutorial.spiders', 'SPIDER_MODULES': ['tutorial.spiders'], 'BOT_NAME': 'tutorial'}
2016-07-08 03:40:40 [scrapy] INFO: Enabled extensions: CloseSpider, TelnetConsole, LogStats, CoreStats, SpiderState
2016-07-08 03:40:40 [scrapy] INFO: Enabled downloader middlewares: HttpAuthMiddleware, DownloadTimeoutMiddleware, UserAgentMiddleware, RetryMiddleware, DefaultHeadersMiddleware, MetaRefreshMiddleware, HttpCompressionMiddleware, RedirectMiddleware, CookiesMiddleware, ChunkedTransferMiddleware, DownloaderStats
2016-07-08 03:40:40 [scrapy] INFO: Enabled spider middlewares: HttpErrorMiddleware, OffsiteMiddleware, RefererMiddleware, UrlLengthMiddleware, DepthMiddleware
2016-07-08 03:40:40 [scrapy] INFO: Enabled item pipelines: DailyStockPipeline
2016-07-08 03:40:40 [scrapy] INFO: Spider opened
2016-07-08 03:40:40 [scrapy] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2016-07-08 03:40:40 [scrapy] DEBUG: Telnet console listening on 127.0.0.1:6023
2016-07-08 03:40:40 [scrapy] DEBUG: Crawled (200) <GET https://www.google.com/finance/historical?q=KRX%3AKOSPI200> (referer: None)
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 244.59999999999999, 'volume': 62210000, 'price_open': 243.15000000000001, 'price_low': 242.63, 'date': datetime.datetime(2016, 7, 7, 0, 0), 'price_high': 245.02000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 241.86000000000001, 'volume': 71672000, 'price_open': 245.37, 'price_low': 240.72999999999999, 'date': datetime.datetime(2016, 7, 6, 0, 0), 'price_high': 245.74000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 246.91, 'volume': 54810000, 'price_open': 247.66999999999999, 'price_low': 246.50999999999999, 'date': datetime.datetime(2016, 7, 5, 0, 0), 'price_high': 247.84}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 247.62, 'volume': 63633000, 'price_open': 246.66, 'price_low': 246.16, 'date': datetime.datetime(2016, 7, 4, 0, 0), 'price_high': 247.94}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 246.52000000000001, 'volume': 63045000, 'price_open': 244.96000000000001, 'price_low': 244.78, 'date': datetime.datetime(2016, 7, 1, 0, 0), 'price_high': 247.56999999999999}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 244.13999999999999, 'volume': 75453000, 'price_open': 244.25, 'price_low': 242.52000000000001, 'date': datetime.datetime(2016, 6, 30, 0, 0), 'price_high': 244.47999999999999}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 242.31999999999999, 'volume': 67355000, 'price_open': 241.28999999999999, 'price_low': 240.68000000000001, 'date': datetime.datetime(2016, 6, 29, 0, 0), 'price_high': 243.71000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 240.08000000000001, 'volume': 79648000, 'price_open': 236.78, 'price_low': 236.72999999999999, 'date': datetime.datetime(2016, 6, 28, 0, 0), 'price_high': 240.46000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 239.28, 'volume': 95450000, 'price_open': 236.78999999999999, 'price_low': 236.68000000000001, 'date': datetime.datetime(2016, 6, 27, 0, 0), 'price_high': 239.28}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 239.21000000000001, 'volume': 190032000, 'price_open': 248.22999999999999, 'price_low': 234.97, 'date': datetime.datetime(2016, 6, 24, 0, 0), 'price_high': 248.27000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 246.31, 'volume': 65983000, 'price_open': 246.41999999999999, 'price_low': 245.63999999999999, 'date': datetime.datetime(2016, 6, 23, 0, 0), 'price_high': 246.81}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 246.75, 'volume': 65848000, 'price_open': 245.16999999999999, 'price_low': 244.78999999999999, 'date': datetime.datetime(2016, 6, 22, 0, 0), 'price_high': 247.03}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 245.34, 'volume': 58402000, 'price_open': 244.59999999999999, 'price_low': 243.99000000000001, 'date': datetime.datetime(2016, 6, 21, 0, 0), 'price_high': 245.5}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 245.16999999999999, 'volume': 75085000, 'price_open': 244.44, 'price_low': 243.74000000000001, 'date': datetime.datetime(2016, 6, 20, 0, 0), 'price_high': 245.63}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 241.63, 'volume': 77334000, 'price_open': 243.34999999999999, 'price_low': 241.31999999999999, 'date': datetime.datetime(2016, 6, 17, 0, 0), 'price_high': 244.02000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 241.61000000000001, 'volume': 83308000, 'price_open': 243.66, 'price_low': 240.52000000000001, 'date': datetime.datetime(2016, 6, 16, 0, 0), 'price_high': 244.0}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 243.30000000000001, 'volume': 84095000, 'price_open': 243.41, 'price_low': 242.19, 'date': datetime.datetime(2016, 6, 15, 0, 0), 'price_high': 244.16}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 243.34999999999999, 'volume': 107840000, 'price_open': 243.78999999999999, 'price_low': 242.36000000000001, 'date': datetime.datetime(2016, 6, 14, 0, 0), 'price_high': 244.46000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 244.05000000000001, 'volume': 79114000, 'price_open': 246.74000000000001, 'price_low': 243.69999999999999, 'date': datetime.datetime(2016, 6, 13, 0, 0), 'price_high': 246.96000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 248.96000000000001, 'volume': 89452000, 'price_open': 249.86000000000001, 'price_low': 248.63, 'date': datetime.datetime(2016, 6, 10, 0, 0), 'price_high': 249.86000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 250.19, 'volume': 138770000, 'price_open': 250.19, 'price_low': 248.46000000000001, 'date': datetime.datetime(2016, 6, 9, 0, 0), 'price_high': 251.50999999999999}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 250.03999999999999, 'volume': 90733000, 'price_open': 248.24000000000001, 'price_low': 247.68000000000001, 'date': datetime.datetime(2016, 6, 8, 0, 0), 'price_high': 250.03999999999999}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 247.84999999999999, 'volume': 79435000, 'price_open': 245.55000000000001, 'price_low': 245.53999999999999, 'date': datetime.datetime(2016, 6, 7, 0, 0), 'price_high': 247.86000000000001}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 244.38999999999999, 'volume': 86345000, 'price_open': 244.94, 'price_low': 243.66999999999999, 'date': datetime.datetime(2016, 6, 3, 0, 0), 'price_high': 244.94}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 244.18000000000001, 'volume': 78953000, 'price_open': 243.88, 'price_low': 243.31, 'date': datetime.datetime(2016, 6, 2, 0, 0), 'price_high': 244.66}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 243.58000000000001, 'volume': 78835000, 'price_open': 242.69999999999999, 'price_low': 242.47999999999999, 'date': datetime.datetime(2016, 6, 1, 0, 0), 'price_high': 244.19}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 243.63, 'volume': 117966000, 'price_open': 241.03, 'price_low': 240.36000000000001, 'date': datetime.datetime(2016, 5, 31, 0, 0), 'price_high': 243.84}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 241.72999999999999, 'volume': 67446000, 'price_open': 241.94, 'price_low': 240.41, 'date': datetime.datetime(2016, 5, 30, 0, 0), 'price_high': 242.06999999999999}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 241.84999999999999, 'volume': 92225000, 'price_open': 241.25, 'price_low': 240.74000000000001, 'date': datetime.datetime(2016, 5, 27, 0, 0), 'price_high': 242.41}
2016-07-08 03:40:40 [scrapy] DEBUG: Scraped from <200 https://www.google.com/finance/historical?q=KRX%3AKOSPI200>
{'symbol': 'KOSPI', 'price_close': 240.58000000000001, 'volume': 130282000, 'price_open': 241.56, 'price_low': 240.44999999999999, 'date': datetime.datetime(2016, 5, 26, 0, 0), 'price_high': 242.05000000000001}
2016-07-08 03:40:40 [scrapy] INFO: Closing spider (finished)
2016-07-08 03:40:40 [scrapy] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 247,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 8127,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2016, 7, 8, 3, 40, 40, 723470),
 'item_scraped_count': 30,
 'log_count/DEBUG': 32,
 'log_count/INFO': 7,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2016, 7, 8, 3, 40, 40, 271766)}
2016-07-08 03:40:40 [scrapy] INFO: Spider closed (finished)
```

크롤링이 완료되면 다음과 같이 sqlite 데이터베이스를 확인할 수 있다.

```
$ sqlite3 dailystock.sqlite 'select * from dailystock'
KOSPI|2016-07-07 00:00:00|243.15|245.02|242.63|244.6|62210000
KOSPI|2016-07-06 00:00:00|245.37|245.74|240.73|241.86|71672000
KOSPI|2016-07-05 00:00:00|247.67|247.84|246.51|246.91|54810000
KOSPI|2016-07-04 00:00:00|246.66|247.94|246.16|247.62|63633000
KOSPI|2016-07-01 00:00:00|244.96|247.57|244.78|246.52|63045000
KOSPI|2016-06-30 00:00:00|244.25|244.48|242.52|244.14|75453000
KOSPI|2016-06-29 00:00:00|241.29|243.71|240.68|242.32|67355000
KOSPI|2016-06-28 00:00:00|236.78|240.46|236.73|240.08|79648000
KOSPI|2016-06-27 00:00:00|236.79|239.28|236.68|239.28|95450000
KOSPI|2016-06-24 00:00:00|248.23|248.27|234.97|239.21|190032000
KOSPI|2016-06-23 00:00:00|246.42|246.81|245.64|246.31|65983000
KOSPI|2016-06-22 00:00:00|245.17|247.03|244.79|246.75|65848000
KOSPI|2016-06-21 00:00:00|244.6|245.5|243.99|245.34|58402000
KOSPI|2016-06-20 00:00:00|244.44|245.63|243.74|245.17|75085000
KOSPI|2016-06-17 00:00:00|243.35|244.02|241.32|241.63|77334000
KOSPI|2016-06-16 00:00:00|243.66|244.0|240.52|241.61|83308000
KOSPI|2016-06-15 00:00:00|243.41|244.16|242.19|243.3|84095000
KOSPI|2016-06-14 00:00:00|243.79|244.46|242.36|243.35|107840000
KOSPI|2016-06-13 00:00:00|246.74|246.96|243.7|244.05|79114000
KOSPI|2016-06-10 00:00:00|249.86|249.86|248.63|248.96|89452000
KOSPI|2016-06-09 00:00:00|250.19|251.51|248.46|250.19|138770000
KOSPI|2016-06-08 00:00:00|248.24|250.04|247.68|250.04|90733000
KOSPI|2016-06-07 00:00:00|245.55|247.86|245.54|247.85|79435000
KOSPI|2016-06-03 00:00:00|244.94|244.94|243.67|244.39|86345000
KOSPI|2016-06-02 00:00:00|243.88|244.66|243.31|244.18|78953000
KOSPI|2016-06-01 00:00:00|242.7|244.19|242.48|243.58|78835000
KOSPI|2016-05-31 00:00:00|241.03|243.84|240.36|243.63|117966000
KOSPI|2016-05-30 00:00:00|241.94|242.07|240.41|241.73|67446000
KOSPI|2016-05-27 00:00:00|241.25|242.41|240.74|241.85|92225000
KOSPI|2016-05-26 00:00:00|241.56|242.05|240.45|240.58|130282000
```